In [2]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import numpy as np
import re
import pickle
from function_library import *
from function_library2 import *

DEBUG:matplotlib.backends:backend module://ipykernel.pylab.backend_inline version unknown


In [8]:
# load people list from pickle
# maybe should save body and title into pickle as well?
l_from = fromPickle("from_list")
l_to   = fromPickle("to_list")
l_cc   = fromPickle("cc_list")

In [5]:
# load df with body
df = pd.read_csv('output4.csv')
df['Sent'] = pd.to_datetime(df['Sent'])
df = df[df['Sent'] > datetime( 2012, 1, 1, 0, 0, 0)]
body = df['Body'].values
subject = df['Subject'].values

In [6]:
# if edison appear in the subject or the body, save the index
edison = 'edison'
idx_list = []
for idx in range(len(body)):
    if edison in body[idx].lower() or edison in subject[idx].lower():
        idx_list.append(idx)

In [10]:
# find all the senders using the index list 
from_list = standardize_triplet(np.array(l_from)[idx_list].tolist())
name_list = set(from_list)

# find all the recipients using the index list 
to_list = standardize_triplet(np.array(l_to)[idx_list].tolist())
for names in to_list:
    name_list |= set(names)
cc_list = standardize_triplet(np.array(l_cc)[idx_list].tolist())
for names in cc_list:
    name_list |= set(names)

# convert to list and sort, in order to save to a txt
name_list = list(name_list)
name_list.sort()

In [11]:
# save to a text file
with open('edison_people_list.txt', 'w') as filehandle:
    for person in name_list[1:]:
        filehandle.write('{} {}: {}\n'.format(person[0], person[1], person[2]))

In [12]:
# number of emails related 
len(idx_list)

1028

In [21]:
# pick out all the related emails and save into another csv
df_edison = df.iloc[idx_list]

del df_edison['Unnamed: 0']
del df_edison['Unnamed: 0.1']

df_edison.to_csv('edison_emails.csv')